<a href="https://colab.research.google.com/github/anas1IA/training_Graph_CNN/blob/main/Training__model_CNN_Graph.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from torch_geometric.loader import DataLoader
import torch
from torch.nn import Linear ,BatchNorm1d
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.nn import global_mean_pool
from torch_geometric.data import Data, InMemoryDataset
import os.path as osp
import torch
import json
import os

In [34]:
!apt-get -qq install -y unrar

In [42]:
!mkdir -p /content/dataset1


In [35]:
!unrar x dataset.rar /content/datasetA/



UNRAR 6.11 beta 1 freeware      Copyright (c) 1993-2022 Alexander Roshal


Extracting from dataset.rar

Creating    /content/datasetA/dataset                                 OK
Creating    /content/datasetA/dataset/processed                       OK
Extracting  /content/datasetA/dataset/processed/data.pt                   24%  OK 
Extracting  /content/datasetA/dataset/processed/pre_filter.pt             24%  OK 
Extracting  /content/datasetA/dataset/processed/pre_transform.pt          24%  OK 
Creating    /content/datasetA/dataset/raw                             OK
Extracting  /content/datasetA/dataset/raw/A.json                          31%  OK 
Extracting  /content/datasetA/dataset/raw/B.json                          35%  OK 
Extracting  /content/datasetA/dataset/raw/C.json                          35%  OK 
Extracting  /content/datasetA/dataset/raw/D.json                          36%  OK 
Extracting  /content/datasetA/da

In [36]:
import os

print(os.listdir('/content/datasetA/'))


['dataset']


In [27]:
pip install torch_geometric

In [64]:


class ASLDataset(InMemoryDataset):

    def __init__(self, root, transform=None, pre_transform=None, pre_filter=None):
        super().__init__(root, transform, pre_transform, pre_filter)
        self.load(self.processed_paths[0])

    @property
    def raw_file_names(self):
        return sorted(os.listdir(self.raw_dir))

    @property
    def processed_file_names(self):
        return ['data.pt']

    def edge_index(self) -> torch.tensor:
        return torch.tensor([[0, 0, 0, 1, 1, 2, 2, 3, 3, 4, 5, 5, 5, 6, 6, 7, 7, 8, 9, 9, 9, 10, 10, 11, 11, 12, 13, 13, 13, 14, 14, 15, 15, 16, 17, 17, 17, 18, 18, 19, 19, 20],
                             [1, 5, 17, 0, 2, 1, 3, 2, 4, 3, 0, 6, 9, 5, 7, 6, 8, 7, 5, 10, 13, 9, 11, 10, 12, 11, 9, 14, 17, 13, 15, 14, 16, 15, 0, 13, 18, 17, 19, 18, 20, 19]]
                            ).type(torch.long)

    def download(self):
        ...

    def process(self):

        data_list = []

        chars = []

        for char in self.raw_file_names:

            jsn = json.load(open(osp.join(self.raw_dir, char), "r"))

            y = torch.tensor(jsn["label"])

            if len(samples := jsn["samples"]) != 0:

                chars.append(char)

                for sample in samples:

                    x = torch.tensor(sample["x"]).type(torch.float32)

                    data = Data(edge_index=self.edge_index(), x=x, y=y)

                    data_list.append(data)


            self.data, self.slices = self.collate(data_list)

            torch.save((self._data, self.slices), self.processed_paths[0])







ASLDataset(6486)


In [ ]:

dataset = ASLDataset(root="datasetA/dataset")
print(dataset)

In [ ]:

# print(len(dataset))
torch.manual_seed(12345)
dataset = dataset

train_dataset = dataset
# test_dataset = dataset[600:]

# print(f'Number of training graphs: {len(train_dataset)}')
# print(f'Number of test graphs: {len(test_dataset)}')

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
# test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# for step, data in enumerate(train_loader):
#     print(f'Step {step + 1}:')
#     print('=======')
#     print(f'Number of graphs in the current batch: {data.num_graphs}')
#     print(data)
#     print()

class GCN(torch.nn.Module):
    def __init__(self, hidden_channels):
        super(GCN, self).__init__()
        torch.manual_seed(12345)
        self.conv1 = GCNConv(dataset.num_node_features, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        self.conv3 = GCNConv(hidden_channels, hidden_channels)
        self.conv4 = GCNConv(hidden_channels, hidden_channels)
        self.lin = Linear(hidden_channels, dataset.num_classes)
        # self.sigmoid = torch.nn.Sigmoid()

    def forward(self, x, edge_index, batch):
        # 1. Obtain node embeddings
        x = self.conv1(x, edge_index)
        x = x.relu()
        x = self.conv2(x, edge_index)
        x = x.relu()
        x = self.conv3(x, edge_index)
        x = x.relu()
        x = self.conv4(x, edge_index)
        # x = x.relu()
        # x = self.conv3(x, edge_index)
        # 2. Readout layer
        x = global_mean_pool(x, batch)  # [batch_size, hidden_channels]

        # 3. Apply a final classifier
        # x = F.dropout(x, p=0.4, training=self.training)
        x = self.lin(x)


        return x

# model = GCN(hidden_channels=64)
# print(model)
model = GCN(hidden_channels=128)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.CrossEntropyLoss()

def train():
    model.train()

    for data in train_loader:  # Iterate in batches over the training dataset.
         out = model(data.x, data.edge_index, data.batch)  # Perform a single forward pass.
         loss = criterion(out, data.y)  # Compute the loss.
         loss.backward()  # Derive gradients.
         optimizer.step()  # Update parameters based on gradients.
         optimizer.zero_grad()  # Clear gradients.

def test(loader):
     model.eval()

     correct = 0
     for data in loader:  # Iterate in batches over the training/test dataset.
         out = model(data.x, data.edge_index, data.batch)
         pred = out.argmax(dim=1)  # Use the class with highest probability.
         correct += int((pred == data.y).sum())  # Check against ground-truth labels.
     return correct / len(loader.dataset)  # Derive ratio of correct predictions.


for epoch in range(1, 400):
    train()
    train_acc = test(train_loader)
    # test_acc = test(test_loader)
    print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}')


Epoch: 001, Train Acc: 0.1041
Epoch: 002, Train Acc: 0.1101
Epoch: 003, Train Acc: 0.1113
Epoch: 004, Train Acc: 0.1135
Epoch: 005, Train Acc: 0.1213
Epoch: 006, Train Acc: 0.1142
Epoch: 007, Train Acc: 0.1292
Epoch: 008, Train Acc: 0.1343
Epoch: 009, Train Acc: 0.1331
Epoch: 010, Train Acc: 0.1206
Epoch: 011, Train Acc: 0.1363
Epoch: 012, Train Acc: 0.1391
Epoch: 013, Train Acc: 0.1369
Epoch: 014, Train Acc: 0.1304
Epoch: 015, Train Acc: 0.1422
Epoch: 016, Train Acc: 0.1429
Epoch: 017, Train Acc: 0.1378
Epoch: 018, Train Acc: 0.1409
Epoch: 019, Train Acc: 0.1411
Epoch: 020, Train Acc: 0.1438
Epoch: 021, Train Acc: 0.1496
Epoch: 022, Train Acc: 0.1506
Epoch: 023, Train Acc: 0.1548
Epoch: 024, Train Acc: 0.1406
Epoch: 025, Train Acc: 0.1619
Epoch: 026, Train Acc: 0.1355
Epoch: 027, Train Acc: 0.1355
Epoch: 028, Train Acc: 0.1459
Epoch: 029, Train Acc: 0.1594
Epoch: 030, Train Acc: 0.1474
Epoch: 031, Train Acc: 0.1468
Epoch: 032, Train Acc: 0.1496
Epoch: 033, Train Acc: 0.1511
Epoch: 034